<a target="_blank" href="https://colab.research.google.com/github/eeg2025/startkit/blob/main/challenge_2.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Challenge 2: Predicting the externalizing factor from EEG

The externalizing factor is a widely recognized construct in mental health research, representing a common underlying dimension of psychopathology across various disorders.
Currently, the externalizing factor is often assessed using self-report questionnaires or clinician ratings, which can be subjective, prone to bias, and time-consuming.
**The Challenge 2** consists of developing a model to predict the externalizing factor from EEG recordings.

The challenge encourages learning physiologically meaningful signal representations and discovery of reproducible biomarkers.
Models of any size should emphasize robust, interpretable features that generalize across subjects,
sessions, and acquisition sites.

Unlike a standard in-distribution classification task, this regression problem stresses out-of-distribution robustness
and extrapolation. The goal is not only to minimize error on seen subjects, but also to transfer effectively to unseen data.

Ensure the dataset is available locally. If not, see the [dataset download guide](https://eeg2025.github.io/data/#downloading-the-data)

## Contents of this start kit

0. Understand the externalizing factor regression task.
1. Loading the data.
2. Wrap the data into a PyTorch-compatible dataset.
3. Define, train and save a model.

> **Note:** If you need additional explanations on the [`EEGChallengeDataset`](https://eeglab.org/EEGDash/api/eegdash.html#eegdash.EEGChallengeDataset) class, dataloading, [`braindecode`](https://braindecode.org/stable/models/models_table.html)'s deep learning models, or brain decoding in general, please refer to the start-kit of challenge 1 which delves deeper into these topics.

More contents will be released during the competition inside the [`eegdash`](https://eeglab.org/EEGDash/overview.html) [examples webpage](https://eeglab.org/EEGDash/generated/auto_examples/index.html).

We will establish direct communication channels with you over the next two months, resolve platform issues, and discuss other details.

We are going to do weekly one-hour support time to help teams that may have difficulties with any technical aspects that may arise.

We will have closer communication starting next week, 08/09/2025.

Before we begin, I just want to make a deal with you, ok?

This is a community competition with a strong open-source foundation. When I say open-source, I mean volunteer work.

So, if you see something that does not work or could be improved, first, **please be kind**, and we will fix it together on GitHub, okay?

The entire decoding community will only go further when we stop solving the same problems over and over again, and start working together.

----
| The tutorial assumes some prior knowledge of deep learning concepts and EEG concepts:|
| --- |
|* Standard neural network architectures, e.g., convolutional neural networks|
|* Optimization by batch gradient descent and backpropagation|
|* Overfitting, early stopping, regularisation |
|* Some knowledge of pytorch and, optionally, of the pytorch Lightning framework|
|* That you know what EEG is |
|* That you have basic familiarity with EEG preprocessing |
|* Like and support open-source :) |
----

In [1]:
!pip install braindecode
!pip install eegdash


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [2]:
from pathlib import Path
import math
import os
import random
from joblib import Parallel, delayed

import torch
from torch.utils.data import DataLoader
from torch import optim
from torch.nn.functional import l1_loss
from braindecode.preprocessing import create_fixed_length_windows
from braindecode.datasets.base import EEGWindowsDataset, BaseConcatDataset, BaseDataset
from braindecode.models import EEGNeX
from eegdash import EEGChallengeDataset

⚠️ **In case of colab, before starting, make sure you're on a GPU instance for faster training!** ⚠️

> If running on Google Colab, please request a GPU runtime by clicking `Runtime/Change runtime type` in the top bar menu, then selecting 'T4 GPU' under 'Hardware accelerator'.

In [3]:
# CORRECTED: Use your actual data directory
DATA_DIR = Path("data") 

# Check if data directory exists
if not DATA_DIR.exists():
    print(f"Data directory not found: {DATA_DIR}")
    print("Make sure you have downloaded the data to the data/ folder")
    raise FileNotFoundError(f"Data directory not found: {DATA_DIR}")

print(f"📁 Using data directory: {DATA_DIR.resolve()}")

# Check what releases are available
available_releases = [d.name for d in DATA_DIR.iterdir() if d.is_dir() and d.name.startswith("release_")]
print(f"📊 Available releases: {available_releases}")

# We define the list of releases to load.
# Here, all but release 5 is loaded
release_list = ["R1", "R2", "R3", "R4", "R6", "R7", "R8", "R9", "R10", "R11"]

all_datasets_list = [
    EEGChallengeDataset(
        release=release,
        task="contrastChangeDetection",
        mini=False,  # Use full dataset, not mini
        description_fields=[
            "subject",
            "session",
            "run",
            "task",
            "age",
            "gender",
            "sex",
            "p_factor",
            "externalizing",  # Add externalizing field for Challenge 2
        ],
        cache_dir=DATA_DIR,
    )
    for release in release_list
]
print("Datasets loaded")
sub_rm = ["NDARWV769JM7", "NDARME789TD2", "NDARUA442ZVF", "NDARJP304NK1",
          "NDARTY128YLU", "NDARDW550GU6", "NDARLD243KRE", "NDARUJ292JXV", "NDARBA381JGH"]


📁 Using data directory: /Users/kimberly/Documents/ESE5380/eeg_challenge/data
📊 Available releases: ['release_8', 'release_1', 'release_6', 'release_7', 'release_9', 'release_5', 'release_2', 'release_3', 'release_4', 'release_11', 'release_10']


/Users/kimberly/Documents/ESE5380/eeg_challenge/eeg_env/lib/python3.13/site-packages/eegdash/dataset/dataset.py:126: UserWarning: 

[EEGChallengeDataset] EEG 2025 Competition Data Notice:
-------------------------------------------------------
This object loads the HBN dataset that has been preprocessed for the EEG Challenge:
  - Downsampled from 500Hz to 100Hz
  - Bandpass filtered (0.5–50 Hz)

For full preprocessing details, see:
  https://github.com/eeg2025/downsample-datasets

IMPORTANT: The data accessed via `EEGChallengeDataset` is NOT identical to what you get from `EEGDashDataset` directly.
If you are participating in the competition, always use `EEGChallengeDataset` to ensure consistency with the challenge data.


  warn(
/Users/kimberly/Documents/ESE5380/eeg_challenge/eeg_env/lib/python3.13/site-packages/eegdash/dataset/dataset.py:126: UserWarning: 

[EEGChallengeDataset] EEG 2025 Competition Data Notice:
-------------------------------------------------------
This object loa

Datasets loaded


In [4]:
# # OVERRIDE: Use your local data (NO DOWNLOADING!)
# # This cell overrides the previous cell that was trying to download data

# DATA_DIR = Path("data")  # Use your actual data directory

# # Check if data directory exists
# if not DATA_DIR.exists():
#     print(f"❌ Data directory not found: {DATA_DIR}")
#     print("💡 Make sure you have downloaded the data to the data/ folder")
#     raise FileNotFoundError(f"Data directory not found: {DATA_DIR}")

# print(f"📁 Using data directory: {DATA_DIR.resolve()}")

# # Check what releases are available
# available_releases = [d.name for d in DATA_DIR.iterdir() if d.is_dir() and d.name.startswith("release_")]
# print(f"📊 Available releases: {available_releases}")

# # Load ONLY R5 to avoid downloading
# release_list = ["R5"]

# all_datasets_list = [
#     EEGChallengeDataset(
#         release=release,
#         task="contrastChangeDetection",
#         mini=False,  # Use full dataset
#         description_fields=[
#             "subject",
#             "session",
#             "run",
#             "task",
#             "age",
#             "gender",
#             "sex",
#             "p_factor",
#             "externalizing",  # Required for Challenge 2
#         ],
#         cache_dir=DATA_DIR,
#     )
#     for release in release_list
# ]
# print("✅ Datasets loaded from local data (NO DOWNLOADING)")
# sub_rm = ["NDARWV769JM7", "NDARME789TD2", "NDARUA442ZVF", "NDARJP304NK1",
#           "NDARTY128YLU", "NDARDW550GU6", "NDARLD243KRE", "NDARUJ292JXV", "NDARBA381JGH"]


In [5]:
# # UPDATED: Use your existing local data structure
# DATA_DIR = Path("eeg_cache")  # Use your existing cache directory

# # Check if cache directory exists
# if not DATA_DIR.exists():
#     print(f"Cache directory not found: {DATA_DIR}")
#     print("💡 Run your challenge_1.ipynb first to cache the data")
#     DATA_DIR = Path("data")  # Fallback to data directory
#     print(f"🔄 Using fallback directory: {DATA_DIR}")

# print(f"📁 Using data directory: {DATA_DIR.resolve()}")

# # We define the list of releases to load.
# # Here, only release 5 is loaded.
# release_list = ["R5"]

# all_datasets_list = [
#     EEGChallengeDataset(
#         release=release,
#         task="contrastChangeDetection",
#         mini=False,  # Use full dataset, not mini
#         description_fields=[
#             "subject",
#             "session",
#             "run",
#             "task",
#             "age",
#             "gender",
#             "sex",
#             "p_factor",
#             "externalizing",  # Add externalizing field for Challenge 2
#         ],
#         cache_dir=DATA_DIR,
#     )
#     for release in release_list
# ]
# print("Datasets loaded")
# sub_rm = ["NDARWV769JM7", "NDARME789TD2", "NDARUA442ZVF", "NDARJP304NK1",
#           "NDARTY128YLU", "NDARDW550GU6", "NDARLD243KRE", "NDARUJ292JXV", "NDARBA381JGH"]


In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
if device == "cuda":
    msg = "CUDA-enabled GPU found. Training should be faster."
else:
    msg = (
        "No GPU found. Training will be carried out on CPU, which might be "
        "slower.\n\nIf running on Google Colab, you can request a GPU runtime by"
        " clicking\n`Runtime/Change runtime type` in the top bar menu, then "
        "selecting 'T4 GPU'\nunder 'Hardware accelerator'."
    )
print(msg)

No GPU found. Training will be carried out on CPU, which might be slower.

If running on Google Colab, you can request a GPU runtime by clicking
`Runtime/Change runtime type` in the top bar menu, then selecting 'T4 GPU'
under 'Hardware accelerator'.


## 0. Understanding the externalizing factor regression task.

The externalizing factor is a widely recognized construct in mental health research, representing a common underlying dimension of psychopathology across various disorders.
Currently, the externalizing factor is often assessed using self-report questionnaires or clinician ratings, which can be subjective, prone to bias, and time-consuming.
In the dataset of this challenge, the externalizing factor was assessed using the Child Behavior Checklist (CBCL) [McElroy et al., (2017)](https://doi.org/10.1111/jcpp.12849).

The goal of Challenge 2 is to develop a model to predict the externalizing factor from EEG recordings.
**The feasibility of using EEG data for this purpose is still an open question**.
The solution may involve finding meaningful representations of the EEG data that correlate with the externalizing factor scores.
The challenge encourages learning physiologically meaningful signal representations and discovery of reproducible biomarkers.
If contestants are successful in this task, it could pave the way for more objective and efficient assessments of the externalizing factor in clinical settings.

## 1. Loading the data

### A. Define local path and (down)load the data

In this challenge 2 example, we load the EEG 2025 release using EEG Dash.

> **Note:** in this example notebook, we load the contrast change detection task from one mini release only as an example. Naturally, you are encouraged to train your models on all complete releases, using data from all the tasks you deem relevant.

In [7]:
# # The first step is to define the cache folder!
# DATA_DIR = Path("~/mne_data/eeg2025_competition")

# # Creating the path if it does not exist
# DATA_DIR.mkdir(parents=True, exist_ok=True)

# # We define the list of releases to load.
# # Here, only release 5 is loaded.
# release_list = ["R1", "R2", "R3", "R4", "R6", "R7", "R8", "R9", "R10", "R11"]

# all_datasets_list = [
#     EEGChallengeDataset(
#         release=release,
#         task="contrastChangeDetection",
#         mini=True,
#         description_fields=[
#             "subject",
#             "session",
#             "run",
#             "task",
#             "age",
#             "gender",
#             "sex",
#             "p_factor",
#         ],
#         cache_dir=DATA_DIR,
#     )
#     for release in release_list
# ]
# print("Datasets loaded")
# sub_rm = ["NDARWV769JM7", "NDARME789TD2", "NDARUA442ZVF", "NDARJP304NK1",
#           "NDARTY128YLU", "NDARDW550GU6", "NDARLD243KRE", "NDARUJ292JXV", "NDARBA381JGH"]

### B. Combine the datasets into a single one

Here, we combine the datasets from the different releases into a single
``BaseConcatDataset`` object.

In [8]:
from pathlib import Path
from eegdash import EEGChallengeDataset
from braindecode.datasets.base import BaseConcatDataset

DATA_DIR = Path("data")
release_list = ["R5"]  # adjust if needed

# Map releases to expected folder names (full dataset, not mini)
release_to_ds = {
    "R1": "ds005505-bdf", "R2": "ds005506-bdf", "R3": "ds005507-bdf",
    "R4": "ds005508-bdf", "R5": "ds005509-bdf", "R6": "ds005510-bdf",
    "R7": "ds005511-bdf", "R8": "ds005512-bdf", "R9": "ds005513-bdf",
    "R10": "ds005514-bdf", "R11": "ds005515-bdf",
}

# Hard checks: correct folder and mini/full match
for r in release_list:
    rn = r[1:]
    release_dir = DATA_DIR / f"release_{rn}"
    ds_dir = release_dir / release_to_ds[r]
    assert release_dir.exists(), f"Missing folder: {release_dir}"
    assert ds_dir.exists(), f"Missing dataset folder: {ds_dir} (mini flag mismatch?)"

# IMPORTANT: point cache_dir to the per-release folder you actually have
all_datasets_list = [
    EEGChallengeDataset(
        release=r,
        task="contrastChangeDetection",
        mini=False,               # set True ONLY if you have -bdf-mini folders
        description_fields=[
            "subject","session","run","task","age","gender","sex","p_factor","externalizing"
        ],
        cache_dir=DATA_DIR / f"release_{r[1:]}",  # <- critical
        download=False,           # never fetch
    )
    for r in release_list
]

all_datasets = BaseConcatDataset(all_datasets_list)

/Users/kimberly/Documents/ESE5380/eeg_challenge/eeg_env/lib/python3.13/site-packages/eegdash/dataset/dataset.py:126: UserWarning: 

[EEGChallengeDataset] EEG 2025 Competition Data Notice:
-------------------------------------------------------
This object loads the HBN dataset that has been preprocessed for the EEG Challenge:
  - Downsampled from 500Hz to 100Hz
  - Bandpass filtered (0.5–50 Hz)

For full preprocessing details, see:
  https://github.com/eeg2025/downsample-datasets

IMPORTANT: The data accessed via `EEGChallengeDataset` is NOT identical to what you get from `EEGDashDataset` directly.
If you are participating in the competition, always use `EEGChallengeDataset` to ensure consistency with the challenge data.


  warn(


In [9]:
raws = Parallel(n_jobs=os.cpu_count())(
    delayed(lambda d: d.raw)(d) for d in all_datasets.datasets
)

In [10]:
# all_datasets = BaseConcatDataset(all_datasets_list)
# print(all_datasets.description)

# raws = Parallel(n_jobs=os.cpu_count())(
#     delayed(lambda d: d.raw)(d) for d in all_datasets.datasets
# )

In [ ]:
# # Use existing train/validation splits (created in Cell 15) instead of recreating them!
# # The train_windows_ds and val_windows_ds are already BaseConcatDatasets ready to use.

# print("✅ Using existing train/validation splits from Cell 15")
# print(f"   Train windows: {len(train_windows_ds)}")
# print(f"   Validation windows: {len(val_windows_ds)}")

# # Assign for grid search - these work directly with DataLoader
# train_dataset = train_windows_ds
# val_dataset = val_windows_ds

# print("✅ Dataset preparation complete - ready for grid search!")

# # NOTE: If you see a cell below that converts windows_ds to lists and splits it again,
# # you can skip/comment out that cell since we're using the splits created in Cell 15.


✅ Using existing train/validation splits from Cell 15


NameError: name 'train_windows_ds' is not defined

### C. Inspect your data

We can check what is inside the dataset consuming the
MNE-object inside the Braindecode dataset.

The following snippet, if uncommented, will show the first 10 seconds of the raw EEG signal.
We can also inspect the data further by looking at the events and annotations.
We strongly recommend you to take a look into the details and check how the events are structured.

In [12]:
# raw = all_datasets.datasets[0].raw  # mne.io.Raw object
# print(raw.info)

# raw.plot(duration=10, scalings="auto", show=True)

# print(raw.annotations)

SFREQ = 100

## 2. Wrap the data into a PyTorch-compatible dataset

The class below defines a dataset wrapper that will extract 2-second windows,
uniformly sampled over the whole signal. In addition, it will add useful information
about the extracted windows, such as the externalizing factor, the subject or the task.

In [13]:
class DatasetWrapper(BaseDataset):
    def __init__(
        self,
        dataset: EEGWindowsDataset,
        crop_size_samples: int,
        target_name: str = "externalizing",
        seed=None,
    ):
        self.dataset = dataset
        self.crop_size_samples = crop_size_samples
        self.target_name = target_name
        self.rng = random.Random(seed)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        X, _, crop_inds = self.dataset[index]

        target = self.dataset.description[self.target_name]
        target = float(target)

        # Additional information:
        infos = {
            "subject": self.dataset.description["subject"],
            "sex": self.dataset.description["sex"],
            "age": float(self.dataset.description["age"]),
            "task": self.dataset.description["task"],
            "session": self.dataset.description.get("session", None) or "",
            "run": self.dataset.description.get("run", None) or "",
        }

        # Randomly crop the signal to the desired length:
        i_window_in_trial, i_start, i_stop = crop_inds
        assert i_stop - i_start >= self.crop_size_samples, f"{i_stop=} {i_start=}"
        start_offset = self.rng.randint(0, i_stop - i_start - self.crop_size_samples)
        i_start = i_start + start_offset
        i_stop = i_start + self.crop_size_samples
        X = X[:, start_offset : start_offset + self.crop_size_samples]

        return X, target, (i_window_in_trial, i_start, i_stop), infos

We filter out certain recordings, create fixed length windows and finally make use of our `DatasetWrapper`.

In [14]:
# Filter out recordings that are too short
all_datasets = BaseConcatDataset(
    [
        ds
        for ds in all_datasets.datasets
        if not ds.description.subject in sub_rm
        and ds.raw.n_times >= 4 * SFREQ
        and len(ds.raw.ch_names) == 129
        and not math.isnan(ds.description["p_factor"])
    ]
)

# Create 4-seconds windows with 2-seconds stride
windows_ds = create_fixed_length_windows(
    all_datasets,
    window_size_samples=4 * SFREQ,
    window_stride_samples=2 * SFREQ,
    drop_last_window=True,
)

# Wrap each sub-dataset in the windows_ds
windows_ds = BaseConcatDataset(
    [DatasetWrapper(ds, crop_size_samples=2 * SFREQ) for ds in windows_ds.datasets]
)

## 3. Define, train and save a model

Now we have our pytorch dataset necessary for the training!

Below, we define a simple EEGNeX model from Braindecode.
All the braindecode models expect the input to be of shape (batch_size, n_channels, n_times)
and have a test coverage about the behavior of the model.
However, you can use any pytorch model you want.

In [21]:
# Initialize model
from braindecode.models import EEGConformer
model = EEGConformer(n_chans=129, n_outputs=1, n_times=2 * SFREQ).to(device)

# Specify optimizer
optimizer = optim.Adamax(params=model.parameters(), lr=0.00005)

print(model)

/Users/kimberly/Documents/ESE5380/eeg_challenge/eeg_env/lib/python3.13/site-packages/braindecode/models/eegconformer.py:227: UserWarning: This model has only been tested on no more than 64 channels. no guarantee to work with more channels.
  warnings.warn(


Layer (type (var_name):depth-idx)                                      Input Shape               Output Shape              Param #                   Kernel Shape
EEGConformer (EEGConformer)                                            [1, 129, 200]             [1, 1]                    --                        --
├─_PatchEmbedding (patch_embedding): 1-1                               [1, 1, 129, 200]          [1, 7, 40]                --                        --
│    └─Sequential (shallownet): 2-1                                    [1, 1, 129, 200]          [1, 40, 1, 7]             --                        --
│    │    └─Conv2d (0): 3-1                                            [1, 1, 129, 200]          [1, 40, 129, 176]         1,040                     [1, 25]
│    │    └─Conv2d (1): 3-2                                            [1, 40, 129, 176]         [1, 40, 1, 176]           206,440                   [129, 1]
│    │    └─BatchNorm2d (2): 3-3                                   

Finally, we can train our model. Here we define a simple training loop using pure PyTorch.

> In this example, we only train for a single epoch. Feel free to increase the number of epochs.

In [ ]:
# Create PyTorch Dataloader

# Note: On macOS with spawn start method, custom classes defined in-notebook
# are not picklable by worker processes. Use a single-process loader here.
num_workers = 0

dataloader = DataLoader(
    windows_ds,
    batch_size=128,
    shuffle=True,
    num_workers=num_workers,
)

n_epochs = 10

# Train model for 1 epoch
for epoch in range(n_epochs):

    for idx, batch in enumerate(dataloader):
        # Reset gradients
        optimizer.zero_grad()

        # Unpack the batch
        X, y, crop_inds, infos = batch
        X = X.to(dtype=torch.float32, device=device)
        y = y.to(dtype=torch.float32, device=device).unsqueeze(1)

        # Forward pass
        y_pred = model(X)

        # Compute loss
        loss = l1_loss(y_pred, y)
        print(f"Epoch {epoch} - step {idx}, loss: {loss.item()}")

        # Gradient backpropagation
        loss.backward()
        optimizer.step()

# Finally, we can save the model for later use
save_path = "weights_challenge_2_10_29.pt"
torch.save(model.state_dict(), save_path)
print(f"Model saved as '{save_path}'")

Epoch 0 - step 0, loss: 0.5285935401916504
Epoch 0 - step 1, loss: 0.5805569887161255
Epoch 0 - step 2, loss: 0.47330036759376526
Epoch 0 - step 3, loss: 0.5607393383979797
Epoch 0 - step 4, loss: 0.5905691385269165
Epoch 0 - step 5, loss: 0.5398038625717163
Epoch 0 - step 6, loss: 0.6001657843589783
Epoch 0 - step 7, loss: 0.5866872072219849
Epoch 0 - step 8, loss: 0.5800319314002991
Epoch 0 - step 9, loss: 0.6206172704696655
Epoch 0 - step 10, loss: 0.5596029758453369
Epoch 0 - step 11, loss: 0.5080515742301941
Epoch 0 - step 12, loss: 0.5419822931289673
Epoch 0 - step 13, loss: 0.575488269329071
Epoch 0 - step 14, loss: 0.5354633927345276
Epoch 0 - step 15, loss: 0.5948777198791504
Epoch 0 - step 16, loss: 0.5701531171798706
Epoch 0 - step 17, loss: 0.5359181761741638
Epoch 0 - step 18, loss: 0.500827968120575
Epoch 0 - step 19, loss: 0.5515122413635254
Epoch 0 - step 20, loss: 0.5830295085906982
Epoch 0 - step 21, loss: 0.5923418998718262
Epoch 0 - step 22, loss: 0.5777353644371033

In [ ]:
# # Training function with early stopping
# def train_model_with_early_stopping(model, train_loader, val_loader, params, config):
#     """
#     Train a model with early stopping and return best validation metrics
#     """
#     # Create fresh model instance
#     model_copy = EEGNeX(n_chans=129, n_outputs=1, n_times=2 * SFREQ).to(config['device'])
    
#     # Setup optimizer and scheduler
#     optimizer = torch.optim.AdamW(
#         model_copy.parameters(), 
#         lr=params['learning_rate'], 
#         weight_decay=params['weight_decay']
#     )
#     scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
#         optimizer, T_max=params['n_epochs'] - 1
#     )
#     loss_fn = torch.nn.MSELoss()
    
#     # Early stopping setup
#     patience = params['patience']
#     min_delta = config['min_delta']
#     best_rmse = float('inf')
#     epochs_no_improve = 0
#     best_state = None
#     best_epoch = 0
    
#     train_losses = []
#     val_losses = []
#     train_rmses = []
#     val_rmses = []
    
#     for epoch in range(1, params['n_epochs'] + 1):
#         # Training phase
#         model_copy.train()
#         epoch_train_loss = 0.0
#         epoch_train_predictions = []
#         epoch_train_targets = []
        
#         for batch_idx, batch in enumerate(train_loader):
#             X, y, crop_inds, infos = batch
#             X = X.to(dtype=torch.float32, device=config['device'])
#             y = y.to(dtype=torch.float32, device=config['device']).unsqueeze(1)
            
#             optimizer.zero_grad()
#             y_pred = model_copy(X)
#             loss = loss_fn(y_pred, y)
#             loss.backward()
#             optimizer.step()
            
#             epoch_train_loss += loss.item()
#             epoch_train_predictions.extend(y_pred.detach().cpu().numpy().flatten())
#             epoch_train_targets.extend(y.detach().cpu().numpy().flatten())
        
#         scheduler.step()
        
#         # Calculate training metrics
#         avg_train_loss = epoch_train_loss / len(train_loader)
#         train_rmse = np.sqrt(np.mean((np.array(epoch_train_predictions) - np.array(epoch_train_targets))**2))
        
#         # Validation phase
#         model_copy.eval()
#         epoch_val_loss = 0.0
#         epoch_val_predictions = []
#         epoch_val_targets = []
        
#         with torch.no_grad():
#             for batch_idx, batch in enumerate(val_loader):
#                 X, y, crop_inds, infos = batch
#                 X = X.to(dtype=torch.float32, device=config['device'])
#                 y = y.to(dtype=torch.float32, device=config['device']).unsqueeze(1)
                
#                 y_pred = model_copy(X)
#                 loss = loss_fn(y_pred, y)
                
#                 epoch_val_loss += loss.item()
#                 epoch_val_predictions.extend(y_pred.detach().cpu().numpy().flatten())
#                 epoch_val_targets.extend(y.detach().cpu().numpy().flatten())
        
#         # Calculate validation metrics
#         avg_val_loss = epoch_val_loss / len(val_loader)
#         val_rmse = np.sqrt(np.mean((np.array(epoch_val_predictions) - np.array(epoch_val_targets))**2))
        
#         # Store metrics
#         train_losses.append(avg_train_loss)
#         val_losses.append(avg_val_loss)
#         train_rmses.append(train_rmse)
#         val_rmses.append(val_rmse)
        
#         if config['verbose'] and epoch % 5 == 0:
#             print(f"  Epoch {epoch}: Train RMSE: {train_rmse:.6f}, Val RMSE: {val_rmse:.6f}")
        
#         # Early stopping check
#         if val_rmse < best_rmse - min_delta:
#             best_rmse = val_rmse
#             best_state = copy.deepcopy(model_copy.state_dict())
#             best_epoch = epoch
#             epochs_no_improve = 0
#         else:
#             epochs_no_improve += 1
#             if epochs_no_improve >= patience:
#                 if config['verbose']:
#                     print(f"  Early stopping at epoch {epoch}. Best Val RMSE: {best_rmse:.6f}")
#                 break
    
#     # Load best model state
#     if best_state is not None:
#         model_copy.load_state_dict(best_state)
    
#     return {
#         'best_val_rmse': best_rmse,
#         'best_epoch': best_epoch,
#         'final_train_rmse': train_rmses[-1],
#         'final_val_rmse': val_rmses[-1],
#         'train_losses': train_losses,
#         'val_losses': val_losses,
#         'train_rmses': train_rmses,
#         'val_rmses': val_rmses,
#         'model_state': best_state
#     }


In [ ]:
# # Data splitting for train/validation
# from sklearn.model_selection import train_test_split
# import numpy as np

# # Convert dataset to lists for easier splitting
# X_data = []
# y_data = []
# infos_data = []

# print("Preparing data for train/validation split...")
# for i in range(len(windows_ds)):
#     X, y, crop_inds, infos = windows_ds[i]
#     X_data.append(X)
#     y_data.append(y)
#     infos_data.append(infos)

# # Convert to numpy arrays
# X_data = np.array(X_data)
# y_data = np.array(y_data)

# # Split data (80% train, 20% validation)
# X_train, X_val, y_train, y_val = train_test_split(
#     X_data, y_data, test_size=0.2, random_state=42, stratify=None
# )

# print(f"Training samples: {len(X_train)}")
# print(f"Validation samples: {len(X_val)}")

# # Create custom datasets for train/val
# class SimpleDataset(torch.utils.data.Dataset):
#     def __init__(self, X, y):
#         self.X = torch.FloatTensor(X)
#         self.y = torch.FloatTensor(y)
    
#     def __len__(self):
#         return len(self.X)
    
#     def __getitem__(self, idx):
#         return self.X[idx], self.y[idx]

# train_dataset = SimpleDataset(X_train, y_train)
# val_dataset = SimpleDataset(X_val, y_val)

# print("Data preparation complete!")


Preparing data for train/validation split...
Training samples: 85926
Validation samples: 21482
Data preparation complete!


In [ ]:
# # Grid Search Execution
# def run_grid_search():
#     """
#     Execute grid search over all parameter combinations
#     """
#     # Generate all parameter combinations
#     param_names = list(GRID_SEARCH_PARAMS.keys())
#     param_values = list(GRID_SEARCH_PARAMS.values())
#     param_combinations = list(itertools.product(*param_values))
    
#     print(f"Starting grid search with {len(param_combinations)} combinations...")
    
#     # Results storage
#     results = []
#     best_result = None
#     best_val_rmse = float('inf')
    
#     # Create timestamp for this run
#     timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
#     for i, param_combo in enumerate(param_combinations):
#         # Create parameter dictionary
#         params = dict(zip(param_names, param_combo))
        
#         print(f"\n{'='*60}")
#         print(f"Combination {i+1}/{len(param_combinations)}")
#         print(f"Parameters: {params}")
#         print(f"{'='*60}")
        
#         try:
#             # Create data loaders with current batch size
#             train_loader = DataLoader(
#                 train_dataset, 
#                 batch_size=params['batch_size'], 
#                 shuffle=True, 
#                 num_workers=0
#             )
#             val_loader = DataLoader(
#                 val_dataset, 
#                 batch_size=params['batch_size'], 
#                 shuffle=False, 
#                 num_workers=0
#             )
            
#             # Train model
#             result = train_model_with_early_stopping(
#                 model, train_loader, val_loader, params, TRAINING_CONFIG
#             )
            
#             # Add parameter info to result
#             result.update(params)
#             result['combination_id'] = i + 1
#             result['timestamp'] = timestamp
            
#             # Store result
#             results.append(result)
            
#             # Check if this is the best result so far
#             if result['best_val_rmse'] < best_val_rmse:
#                 best_val_rmse = result['best_val_rmse']
#                 best_result = result.copy()
#                 print(f"🎉 NEW BEST! Val RMSE: {best_val_rmse:.6f}")
            
#             print(f"Completed combination {i+1}. Best Val RMSE: {result['best_val_rmse']:.6f}")
            
#         except Exception as e:
#             print(f"❌ Error in combination {i+1}: {str(e)}")
#             # Store error result
#             error_result = {
#                 'combination_id': i + 1,
#                 'timestamp': timestamp,
#                 'error': str(e),
#                 'best_val_rmse': float('inf')
#             }
#             error_result.update(params)
#             results.append(error_result)
    
#     return results, best_result, timestamp

# # Run the grid search
# print("Starting Grid Search...")
# grid_results, best_model_result, run_timestamp = run_grid_search()

# print(f"\n{'='*60}")
# print("GRID SEARCH COMPLETED!")
# print(f"{'='*60}")
# print(f"Best validation RMSE: {best_model_result['best_val_rmse']:.6f}")
# print(f"Best parameters: {dict((k, v) for k, v in best_model_result.items() if k in GRID_SEARCH_PARAMS.keys())}")
# print(f"Best epoch: {best_model_result['best_epoch']}")


Starting Grid Search...
Starting grid search with 729 combinations...

Combination 1/729
Parameters: {'learning_rate': 0.001, 'weight_decay': 0.0001, 'batch_size': 64, 'n_epochs': 10, 'patience': 5, 'dropout_rate': 0.1}
❌ Error in combination 1: not enough values to unpack (expected 4, got 2)

Combination 2/729
Parameters: {'learning_rate': 0.001, 'weight_decay': 0.0001, 'batch_size': 64, 'n_epochs': 10, 'patience': 5, 'dropout_rate': 0.2}
❌ Error in combination 2: not enough values to unpack (expected 4, got 2)

Combination 3/729
Parameters: {'learning_rate': 0.001, 'weight_decay': 0.0001, 'batch_size': 64, 'n_epochs': 10, 'patience': 5, 'dropout_rate': 0.3}
❌ Error in combination 3: not enough values to unpack (expected 4, got 2)

Combination 4/729
Parameters: {'learning_rate': 0.001, 'weight_decay': 0.0001, 'batch_size': 64, 'n_epochs': 10, 'patience': 10, 'dropout_rate': 0.1}
❌ Error in combination 4: not enough values to unpack (expected 4, got 2)

Combination 5/729
Parameters: {'

TypeError: 'NoneType' object is not subscriptable

In [ ]:
# # Results Analysis and Saving
# def analyze_and_save_results(results, best_result, timestamp):
#     """
#     Analyze grid search results and save them
#     """
#     # Convert results to DataFrame
#     df_results = pd.DataFrame(results)
    
#     # Remove columns with lists for cleaner analysis
#     columns_to_remove = ['train_losses', 'val_losses', 'train_rmses', 'val_rmses', 'model_state']
#     df_analysis = df_results.drop(columns=[col for col in columns_to_remove if col in df_results.columns])
    
#     # Sort by best validation RMSE
#     df_analysis = df_analysis.sort_values('best_val_rmse')
    
#     print("\nTop 10 Results:")
#     print("="*80)
#     top_10 = df_analysis.head(10)
#     for idx, row in top_10.iterrows():
#         print(f"Rank {row['combination_id']}: Val RMSE = {row['best_val_rmse']:.6f}")
#         print(f"  Params: LR={row['learning_rate']}, WD={row['weight_decay']}, BS={row['batch_size']}, Epochs={row['best_epoch']}")
#         print()
    
#     # Parameter importance analysis
#     print("\nParameter Analysis:")
#     print("="*50)
    
#     # Group by each parameter and calculate mean RMSE
#     for param in GRID_SEARCH_PARAMS.keys():
#         if param in df_analysis.columns:
#             param_stats = df_analysis.groupby(param)['best_val_rmse'].agg(['mean', 'std', 'count'])
#             print(f"\n{param.upper()}:")
#             print(param_stats)
    
#     # Save results
#     results_filename = f"grid_search_results_{timestamp}.json"
#     df_analysis.to_json(results_filename, orient='records', indent=2)
    
#     # Save best model
#     if best_result and 'model_state' in best_result:
#         best_model_filename = f"best_model_grid_search_{timestamp}.pt"
#         torch.save(best_result['model_state'], best_model_filename)
#         print(f"\nBest model saved as: {best_model_filename}")
    
#     print(f"\nResults saved as: {results_filename}")
    
#     return df_analysis

# # Analyze and save results
# results_df = analyze_and_save_results(grid_results, best_model_result, run_timestamp)


In [ ]:
# # Visualization of Grid Search Results
# def create_visualizations(df_results, timestamp):
#     """
#     Create visualizations for grid search results
#     """
#     plt.style.use('default')
#     fig, axes = plt.subplots(2, 3, figsize=(18, 12))
#     fig.suptitle(f'Grid Search Results Analysis - {timestamp}', fontsize=16, fontweight='bold')
    
#     # 1. RMSE Distribution
#     axes[0, 0].hist(df_results['best_val_rmse'], bins=20, alpha=0.7, color='skyblue', edgecolor='black')
#     axes[0, 0].axvline(df_results['best_val_rmse'].min(), color='red', linestyle='--', linewidth=2, label=f'Best: {df_results["best_val_rmse"].min():.6f}')
#     axes[0, 0].set_xlabel('Validation RMSE')
#     axes[0, 0].set_ylabel('Frequency')
#     axes[0, 0].set_title('Distribution of Validation RMSE')
#     axes[0, 0].legend()
#     axes[0, 0].grid(True, alpha=0.3)
    
#     # 2. Learning Rate vs RMSE
#     lr_rmse = df_results.groupby('learning_rate')['best_val_rmse'].agg(['mean', 'std']).reset_index()
#     axes[0, 1].errorbar(lr_rmse['learning_rate'], lr_rmse['mean'], yerr=lr_rmse['std'], 
#                        marker='o', capsize=5, capthick=2, linewidth=2)
#     axes[0, 1].set_xlabel('Learning Rate')
#     axes[0, 1].set_ylabel('Mean Validation RMSE')
#     axes[0, 1].set_title('Learning Rate vs Validation RMSE')
#     axes[0, 1].set_xscale('log')
#     axes[0, 1].grid(True, alpha=0.3)
    
#     # 3. Weight Decay vs RMSE
#     wd_rmse = df_results.groupby('weight_decay')['best_val_rmse'].agg(['mean', 'std']).reset_index()
#     axes[0, 2].errorbar(wd_rmse['weight_decay'], wd_rmse['mean'], yerr=wd_rmse['std'], 
#                        marker='s', capsize=5, capthick=2, linewidth=2, color='green')
#     axes[0, 2].set_xlabel('Weight Decay')
#     axes[0, 2].set_ylabel('Mean Validation RMSE')
#     axes[0, 2].set_title('Weight Decay vs Validation RMSE')
#     axes[0, 2].set_xscale('log')
#     axes[0, 2].grid(True, alpha=0.3)
    
#     # 4. Batch Size vs RMSE
#     bs_rmse = df_results.groupby('batch_size')['best_val_rmse'].agg(['mean', 'std']).reset_index()
#     axes[1, 0].bar(bs_rmse['batch_size'], bs_rmse['mean'], yerr=bs_rmse['std'], 
#                    capsize=5, alpha=0.7, color='orange', edgecolor='black')
#     axes[1, 0].set_xlabel('Batch Size')
#     axes[1, 0].set_ylabel('Mean Validation RMSE')
#     axes[1, 0].set_title('Batch Size vs Validation RMSE')
#     axes[1, 0].grid(True, alpha=0.3)
    
#     # 5. Epochs vs RMSE (scatter plot)
#     axes[1, 1].scatter(df_results['best_epoch'], df_results['best_val_rmse'], 
#                       alpha=0.6, c=df_results['learning_rate'], cmap='viridis', s=50)
#     axes[1, 1].set_xlabel('Best Epoch')
#     axes[1, 1].set_ylabel('Validation RMSE')
#     axes[1, 1].set_title('Best Epoch vs Validation RMSE\n(Color = Learning Rate)')
#     axes[1, 1].grid(True, alpha=0.3)
    
#     # Add colorbar for the scatter plot
#     cbar = plt.colorbar(axes[1, 1].collections[0], ax=axes[1, 1])
#     cbar.set_label('Learning Rate')
    
#     # 6. Top 10 Results Bar Chart
#     top_10 = df_results.head(10)
#     axes[1, 2].barh(range(len(top_10)), top_10['best_val_rmse'], color='lightcoral', alpha=0.7)
#     axes[1, 2].set_yticks(range(len(top_10)))
#     axes[1, 2].set_yticklabels([f"#{row['combination_id']}" for _, row in top_10.iterrows()])
#     axes[1, 2].set_xlabel('Validation RMSE')
#     axes[1, 2].set_title('Top 10 Best Results')
#     axes[1, 2].grid(True, alpha=0.3)
    
#     # Add RMSE values on bars
#     for i, v in enumerate(top_10['best_val_rmse']):
#         axes[1, 2].text(v + 0.001, i, f'{v:.4f}', va='center', fontsize=8)
    
#     plt.tight_layout()
    
#     # Save the plot
#     plot_filename = f"grid_search_analysis_{timestamp}.png"
#     plt.savefig(plot_filename, dpi=300, bbox_inches='tight')
#     print(f"Visualization saved as: {plot_filename}")
    
#     plt.show()
    
#     return plot_filename

# # Create visualizations
# plot_file = create_visualizations(results_df, run_timestamp)


In [ ]:
# # Optional: Quick Grid Search with Reduced Parameters
# # If you want to run a faster grid search with fewer combinations, uncomment and modify this:

# QUICK_GRID_PARAMS = {
#     'learning_rate': [0.001, 0.002],
#     'weight_decay': [1e-4, 1e-3],
#     'batch_size': [128, 256],
#     'n_epochs': [10, 20],
#     'patience': [5, 10],
# }

# print("Quick Grid Search Parameters (for faster testing):")
# for param, values in QUICK_GRID_PARAMS.items():
#     print(f"  {param}: {values}")
# print(f"Total combinations: {len(list(itertools.product(*QUICK_GRID_PARAMS.values())))}")

# # To run quick grid search, replace GRID_SEARCH_PARAMS with QUICK_GRID_PARAMS above
# # and re-run the grid search cells


Quick Grid Search Parameters (for faster testing):
  learning_rate: [0.001, 0.002]
  weight_decay: [0.0001, 0.001]
  batch_size: [128, 256]
  n_epochs: [10, 20]
  patience: [5, 10]


NameError: name 'itertools' is not defined